# This notebooks generates 4 subclass blobs and recovers them in LRP space

In [35]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import umap

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedShuffleSplit
# Converting labels to 1-Hot Vectors
from sklearn.preprocessing import OneHotEncoder
from mpl_toolkits.mplot3d import Axes3D


import sys
# sys.path.append("/Users/Work/Developer/interpretDL/interprettensor")
root_logdir = "./tf_logs"
data_dir = "data/"
figures_dir = "data/figures/"

# To plot pretty figures
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

np.random.seed(seed=42) 
tf.__version__

'1.13.1'

In [5]:
def get_split_index(features, labels):
    features = np.array(features)
    # The train set will have equal amounts of each target class
    # Performing single split
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    return [[train_index, test_index] for train_index,test_index in split.split(features, labels)]

def plot_history(history):
    plt.close("History")
    fig, axs = plt.subplots(1, 2, figsize=(12,6),num="History")

    # Plot training & validation accuracy values
    axs[0].grid(True)
    axs[0].plot(history.history['binary_accuracy'])
    axs[0].plot(history.history['val_binary_accuracy'])
    axs[0].set(title='Model accuracy', ylabel='Accuracy', xlabel='Epoch')
    axs[0].legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    axs[1].grid(True)
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set(title='Model loss',ylabel='Loss', xlabel='Epoch')
    axs[1].legend(['Train', 'Test'], loc='upper left')

    plt.show()

## Making 4 blobs

In [7]:
from sklearn.datasets.samples_generator import make_blobs
# generate 2d classification dataset
# X, y = make_circles(n_samples=100, noise=0)a
class_size = 200
centers = [(0,0),(1,0),(-1,0),(0,1),(0,-1)]
n_samples = [class_size//(len(centers)-1)]*len(centers)
n_samples[0] = class_size

X, y = make_blobs(n_samples=n_samples, centers=centers, n_features=2, cluster_std=0.1, shuffle=False, random_state=42)

plt.close("Original")
df = pd.DataFrame(dict(x=X[:,0], y=X[:,1], label=y))
fig, ax = plt.subplots(num="Original")
colors = {0:'red', 1:'blue'}
df.plot(ax=ax,kind="scatter", x='x', y='y',c="label", cmap= "Paired")
# plt.colorbar()
plt.show()

modded_labels = y.copy()
modded_samples = df[["x","y"]].copy()
labels = df["label"].copy()
labels[labels > 0] = 1

FigureCanvasNbAgg()

## Seperate out train, test set

In [8]:
train_index, test_index = get_split_index(modded_samples, modded_labels)[0]
features = modded_samples.copy()
X_train = features.iloc[train_index]
y_train = labels.iloc[train_index]
X_test = features.iloc[test_index]
y_test = labels.iloc[test_index]


print("Train Size:", X_train.shape)
print("Test Size:", y_test.shape)


hot_encoder = OneHotEncoder(categories="auto", sparse=False)
hot_encoder.fit(labels.values.reshape(-1,1)) # Since the function expects an array of "features" per sample
print("Categories:", hot_encoder.categories_)
# X_test, y_test.values

Train Size: (320, 2)
Test Size: (80,)
Categories: [array([0, 1])]


In [9]:
NUM_FEATURES = X_train.shape[1]
NUM_LABELS = len(hot_encoder.categories_[0])

### Train a DNN on the modified dataset
> Does not use any normalization

In [14]:
def build_dnn(num_features, num_labels=3):

#     reset_graph()
    
    keras.backend.clear_session()

    nn = keras.models.Sequential()
    Dense = keras.layers.Dense
    
    # Using He initialization
    he_init = tf.keras.initializers.he_uniform()
    
    nn.add(Dense(units = 12, activation="elu", input_dim=num_features,
                kernel_initializer=he_init))
    nn.add(Dense(units = 12, activation="elu",
                kernel_initializer=he_init))
    nn.add(Dense(units=1, activation= "sigmoid",
                kernel_initializer=he_init))

#     BCE = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    
    nn.compile(loss="binary_crossentropy",
                  optimizer='sgd',
                  metrics=['binary_accuracy'])
    
    return nn

def train_model(model, X, y, X_test=[], y_test=[], epochs=30, batch_size=20, verbose=1, plot=True):
    
    ZScaler = []
    callback_list = []
    
    X_train = X
    y_train = y.values
    y_test = y_test.values
    
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size = batch_size,
                        validation_data=(X_test, y_test), callbacks=callback_list, verbose=verbose)
    
#     if plot: plot_history(history)
    
    return history, ZScaler


In [15]:
nn = build_dnn(NUM_FEATURES)
%time history, Zscaler = train_model(nn, X_train, y_train, X_test, y_test, epochs=50, batch_size=20)

Instructions for updating:
Use tf.cast instead.
Train on 320 samples, validate on 80 samples
Epoch 1/50
320/320 [==============================] - 0s 650us/step - loss: 0.8427 - binary_accuracy: 0.4094 - val_loss: 0.7724 - val_binary_accuracy: 0.5500
Epoch 2/50
320/320 [==============================] - 0s 48us/step - loss: 0.7642 - binary_accuracy: 0.4906 - val_loss: 0.7090 - val_binary_accuracy: 0.6250
Epoch 3/50
320/320 [==============================] - 0s 66us/step - loss: 0.7038 - binary_accuracy: 0.5563 - val_loss: 0.6610 - val_binary_accuracy: 0.6875
Epoch 4/50
320/320 [==============================] - 0s 62us/step - loss: 0.6568 - binary_accuracy: 0.6406 - val_loss: 0.6234 - val_binary_accuracy: 0.7500
Epoch 5/50
320/320 [==============================] - 0s 59us/step - loss: 0.6199 - binary_accuracy: 0.6906 - val_loss: 0.5929 - val_binary_accuracy: 0.7500
Epoch 6/50
320/320 [==============================] - 0s 52us/step - loss: 0.5896 - binary_accuracy: 0.8156 - val_loss: 0

In [16]:
# Plotting results from history
plot_history(history)

FigureCanvasNbAgg()

In [17]:
nn.predict(X_test)
_labels = [np.float(x) for x in y_test]
nn.evaluate(X_test, y_test)

80/80 [==============================] - 0s 28us/step


[0.08940797597169876, 1.0]

In [113]:
# sess = tf.Session()
# predictions = nn.predict(Zscaler.transform(modded_samples))
# predictions = nn.predict(modded_samples)
# _labels = [np.float(x) for x in labels]

# sess.run(keras.metrics.binary_accuracy(_labels,predictions.flatten()))
# # predictions[0][0]
# # _labels
# # nn.evaluate(Zscaler.transform(modded_samples),labels.values)
# # nn.evaluate(modded_samples,labels.values)



## Performing SVM on Modded Samples

In [19]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

svm_clf = Pipeline([
#     ("scaler", StandardScaler()),
    ("SVM", SVC(kernel= "rbf", gamma= "scale", C=1))
])

%time svm_clf.fit(X_train, y_train)
svm_clf.score(X_test, y_test)

CPU times: user 2.63 ms, sys: 151 µs, total: 2.79 ms
Wall time: 2.72 ms


1.0

## Performing LRP

In [30]:
model = nn
scaled_samples = X_test
_labels = y_test

predictions = model.predict(scaled_samples)
preds = np.array([np.round(x[0]) for x in predictions])
true_labels = [np.float(x) for x in _labels]


# For test population that was correctly classified
correct = preds == true_labels

# Get correctly predicted samples,labels along with their original labels
original_labels = modded_labels[test_index][correct]
correct_samples = scaled_samples[correct]
correct_labels = _labels[correct]

print("SANITY CHECK")
loss_and_metrics = model.evaluate(correct_samples, correct_labels)
print("Scores on test set: loss={:0.3f} accuracy={:.4f}".format(*loss_and_metrics))

SANITY CHECK
80/80 [==============================] - 0s 34us/step
Scores on test set: loss=0.089 accuracy=1.0000


In [33]:
print("Class Distribution")
pd.Series(original_labels).value_counts()

Class Distribution


0    40
4    10
3    10
2    10
1    10
dtype: int64

In [34]:
import innvestigate
import innvestigate.utils as iutils

def perform_analysis(model, analyzer, data, labels):
    analysis = analyzer.analyze(data)
    prediction = model.predict(data)
    
    df_anal = pd.DataFrame(analysis)
    
    return df_anal


# Stripping the softmax activation from the model
# model_wo_sm = iutils.keras.graph.model_wo_softmax(model)

# Creating an analyzer
lrp_E = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPEpsilon(model=model, epsilon=1e-3)
lrp_Z = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPZPlus(model=model)
lrp_AB   = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPAlpha2Beta1(model=model)

# Getting all the samples that can be correctly predicted
test_idx = correct
all_samples = scaled_samples[test_idx]
all_labels = y_test[test_idx]


# perform_analysis(nn,gradient_analyzer,flowers,types)
all_lrp_AB = perform_analysis(model,lrp_AB, all_samples, all_labels)
all_lrp_E = perform_analysis(model,lrp_E, all_samples, all_labels)
all_lrp_Z = perform_analysis(model,lrp_Z, all_samples, all_labels)


In [37]:
plt.close("Comparison")
fig, axs = plt.subplots(2,2, figsize=(12,10), num="Comparison")
_labels = original_labels

df.plot(ax=axs[0][0],kind="scatter", x='x', y='y',c="label", cmap= "Paired", title="Original Distribution")
all_lrp_E.plot(ax=axs[0][1],kind="scatter", x=0, y=1, c=_labels, cmap="Paired", s=20, alpha=0.75, title="LRP E")

all_lrp_AB.plot(ax=axs[1][0],kind="scatter", x=0, y=1, c=_labels, cmap="Paired", s=20, alpha=0.75, title="LRP AB")
all_lrp_Z.plot(ax=axs[1][1],kind="scatter", x=0, y=1, c=_labels, cmap="Paired", s=20, alpha=0.75, title="LRP Z")

# plt.tight_layout()
plt.savefig(figures_dir+"4 subclass LRP.png")
plt.show()

FigureCanvasNbAgg()